In [12]:
import numpy as np

In [59]:
class FastGrad:
    def __init__(self, gamma, epsilon, n):
        self.gamma = gamma
        self.epsilon = epsilon
        self.n = n
        self.inf = 1e+6
        self.l = 1 / gamma

        self.a = self.alpha = 0
        
        self.u_lambda = self.u_mu = np.ones([n])
        self.y_lambda = self.y_mu = np.zeros(n)
        self.x_lambda = self.x_mu = np.ones([n])
        
        self.x_0 = np.ones([n, n]) / (n**2)

    
    def x_hat(self, c, lambda_y, mu_y):
        n = self.n
        x_hat = self.x_0 * np.exp(
            -(self.gamma + c + lambda_y.repeat(n).reshape(-1, n) + mu_y.repeat(n).reshape(-1, n).T) / self.gamma
        )
        return x_hat / x_hat.sum()
    
    def _new_alpha(self):
        return 1 / (2 * self.l) + np.sqrt(1 / (4 * (self.l**2)) + self.alpha**2)
    
    def _new_a(self, new_alpha):
        return self.a + new_alpha
    
    def _new_y(self, u_lambda, u_mu, x_lambda, x_mu):
        new_alpha = self._new_alpha()
        new_a = self._new_a(new_alpha)
        
        return (new_alpha * u_lambda + self.a * x_lambda) / new_a,\
               (new_alpha * u_mu     + self.a * x_mu)     / new_a
    
    def _new_u(self, c, u_lambda, u_mu, y_lambda_new, y_mu_new):
        x_hat = self.x_hat(c, y_lambda_new, y_mu_new)
        new_alpha = self._new_alpha()
        
        return u_lambda - new_alpha * (p - x_hat.sum(1)),\
               u_mu     - new_alpha * (q - x_hat.sum(0))
    
    def _new_x(self, u_lambda_new, u_mu_new, x_lambda, x_mu):
        new_alpha = self._new_alpha()
        new_a = self._new_a(new_alpha)
        
        return (new_alpha * u_lambda_new + self.a * x_lambda) / new_a,\
               (new_alpha * u_mu_new     + self.a * x_mu)     / new_a
    
    def f(self, c, x):
        return (c * x).sum() + self.gamma * (x * np.log(x / self.x_0)).sum()
    
    def phi(self, c, p, q, lambda_x, mu_x):
        return (lambda_x * p).sum() + (mu_x * q).sum() + \
                self.gamma * np.log(1/np.e * (self.x_0 * np.exp(
                    -(self.gamma + c + lambda_x.repeat(self.n).reshape(-1, self.n) + mu_x.repeat(self.n).reshape(-1, self.n).T) / self.gamma
                )).sum())
    
    def deviation_p_q(self, x, p, q):
        return np.sqrt(np.sum((x.sum(1) - p)**2) + np.sum((x.sum(0) - q)**2))
    
    def fit(self, c, p, q):
        k = 0
        
        while True:
            y_lambda_new, y_mu_new = self._new_y(self.u_lambda, self.u_mu, self.x_lambda, self.x_mu)
            u_lambda_new, u_mu_new = self._new_u(c, self.u_lambda, self.u_mu, y_lambda_new, y_mu_new)
            x_lambda_new, x_mu_new = self._new_x(u_lambda_new, u_mu_new, self.x_lambda, self.x_mu)
            
            self.y_lambda, self.y_mu = (y_lambda_new, y_mu_new)
            self.u_lambda, self.u_mu = (u_lambda_new, u_mu_new)
            self.x_lambda, self.x_mu = (x_lambda_new, x_mu_new)
            
            self.alpha = self._new_alpha()
            self.a     = self._new_a(self.alpha)
            
            x_wave = self.x_hat(c, self.y_lambda, self.y_mu)
            r = np.sqrt((self.x_lambda**2).sum() + (self.x_mu**2).sum())
            epsilon_wave = self.epsilon / r
            
            
            criteria_a = self.deviation_p_q(x_wave, p, q) <= epsilon_wave
            criteria_b = self.f(c, x_wave) - self.phi(c, p, q, self.x_lambda, self.x_mu) <= self.epsilon
            
            if k % 100 == 0:
                print(f"{self.deviation_p_q(x_wave, p, q)}")
                print(f"{self.f(c, x_wave) - self.phi(c, p, q, self.x_lambda, self.x_mu)}\n")
            
            if criteria_a and criteria_b:
                return x_wave, k
            
            k += 1

In [60]:
fgrad = FastGrad(0.5, 0.01, 5)

In [61]:
def sample_batch(n):
    C = np.random.uniform(0, 10, size=[n, n])
    p = np.random.dirichlet(np.ones(n), size=1).ravel()
    q = np.random.dirichlet(np.ones(n), size=1).ravel()
    return C, p, q

In [62]:
c, p, q = sample_batch(5)

In [63]:
fgrad.fit(c, p, q)

0.8605351171669079
4.71063509512352

0.0005901268887826853
8.467929822729062

1.2727575704547177e-05
8.468921003593607

1.7846940146012065e-06
8.468948639424356

3.9978862957250985e-08
8.468944585831254

1.2676606829399784e-08
8.468944466279561

1.0513179985358784e-10
8.468944494495819

1.1248113627058186e-10
8.468944494965575

1.1497820021544223e-12
8.468944494708808

1.0949817433902225e-12
8.468944494708918

3.245362516690806e-14
8.468944494711465

1.1054640192979195e-14
8.468944494711415

7.532775782306874e-16
8.46894449471139

2.7309638792264485e-16
8.46894449471139

5.732047178037375e-16
8.46894449471139

2.9275225451776116e-16
8.46894449471139

5.258930880074457e-16
8.46894449471139

4.714364385389477e-16
8.468944494711392

3.7030556577017743e-16
8.468944494711392

4.573357540805812e-16
8.46894449471139

2.5485679777957566e-16
8.46894449471139

3.0977323880395805e-16
8.46894449471139

4.759595673043547e-16
8.46894449471139

3.1317404493016336e-16
8.46894449471139

3.7834474109809

KeyboardInterrupt: 